In [50]:
%run input/Format.ipynb
import ROOT as root
from array import array
root.gErrorIgnoreLevel = root.kFatal
%jsroot on

/home/yoren/.local/lib/python3.10/site-packages/nbformat/__init__.py:96: MissingIDFieldWarning: Cell is missing an id field, this will become a hard error in future nbformat versions. You may want to use `normalize()` on your notebooks before validations (available since nbformat 5.1.4). Previous versions of nbformat are fixing this issue transparently, and will stop doing so in the future.
  validate(nb)


In [51]:
shift = 0
antiproton = 0

In [52]:
N_centr = 6
colors=[1,2,4,root.kGreen+2,root.kMagenta,root.kOrange+4,root.kGray,root.kCyan]
central_bins = [0,100,0,20,20,40,40,60,60,80,80,100]
pt_boarders = [0.5,0.9,0.2,0.5,0.5,1.0,1.0,5.0,0.5,5]
file_path="input/"
file_names=["my-m_ee_Run14AuAu_46th_new_19619_695runs.root"] #m_ee_Run14AuAu_46th_new_19619_90runs test_BG_v94_0
sim_file_path="../sim/output/Newembed/sngl/"
primary_file_names=['phi_50M_v6.root']
hadron_file_names=['charged_pi_100M_v1.root']
dalitz_file_names=['dalitz_50M_v1.root']
conv_file_names=['conv_100M.root']
sim_file_names = [dalitz_file_names[0]]
legend_names = ["pos","neg","sim_pos","sim_neg"] 
hist_select_3D_names = ["ep_hist","n0_hist","prob_hist","disp_hist","chi2npe0_hist","ttof","temc",\
                        "el_had_dphi","el_had_dz","el_had_dr",'disp_hist_el']

In [53]:
N_hists = len(hist_select_3D_names)

In [54]:
hists_read, hists_sim = [], []

for iFile in range(len(file_names)):
    print(file_path+file_names[iFile])
    infile = root.TFile.Open(file_path+file_names[iFile], "read")
    hists_read_file = []
    for ihist in range(N_hists):
        hists_read_file.append(infile.Get(hist_select_3D_names[ihist]))
        hists_read_file[-1].SetDirectory(root.nullptr)
        hists_read_file[-1].SetName(hists_read_file[-1].GetName()+f"{iFile}")
    hists_read.append(hists_read_file)


for iFile in range(len(sim_file_names)):
    print(sim_file_path+sim_file_names[iFile])
    infile = root.TFile.Open(sim_file_path+sim_file_names[iFile], "read")
    hists_sim_file = []
    for ihist in range(N_hists):
        hists_sim_file.append(infile.Get(hist_select_3D_names[ihist]))
        hists_sim_file[-1].SetDirectory(root.nullptr)
        hists_sim_file[-1].SetName(hists_sim_file[-1].GetName()+f"_sim_{iFile}")
    hists_sim.append(hists_sim_file)

infile.Close()

input/my-m_ee_Run14AuAu_46th_new_19619_695runs.root
../sim/output/Newembed/sngl/dalitz_50M_v1.root


In [55]:
if not shift: 
    antiproton = antiproton*15
    if antiproton>0: 
        shift = 5
        legend_names = ["antip","neg","sim_antip","sim_neg"] 
else: antiproton = 0

In [56]:
type_loc = 0
pt = [1.95,2.45]
c0 = root.TCanvas(f"c0",f"c0",1450,900)
c0.Divide(3,2)
projy = hists_read[0][type_loc].ProjectionY(f"projy{type_loc}")
projz = hists_read[0][type_loc].ProjectionZ(f"projz{type_loc}")
legends = []
for icentr in range(N_centr):
    c0.cd(1+icentr)
    ibiny0 = projy.FindBin(pt[0])
    ibiny1 = projy.FindBin(pt[1])-1
    ibinz0 = shift+projz.FindBin(central_bins[2*icentr+0])
    ibinz1 = shift+projz.FindBin(central_bins[2*icentr+1])-1
    pos_proj = hists_read[iFile][type_loc].ProjectionX(hists_read[iFile][type_loc].GetName()+f"pos_proj_{icentr}",ibiny0,ibiny1,ibinz0,ibinz1)
    Format_Hist_total(pos_proj,title_x="E/p")
    neg_proj = hists_read[iFile][type_loc].ProjectionX(hists_read[iFile][type_loc].GetName()+f"neg_proj_{icentr}",ibiny0,ibiny1,ibinz0+5+antiproton,ibinz1+5+antiproton)
    Format_Hist_total(neg_proj,Lcolor=2)
    sim_pos_proj = hists_sim[iFile][type_loc].ProjectionX(hists_sim[iFile][type_loc].GetName()+f"sim_pos_proj_{icentr}",ibiny0,ibiny1,ibinz0,ibinz1)
    Format_Hist_total(sim_pos_proj,Lcolor=root.kGreen+2)
    sim_neg_proj = hists_sim[iFile][type_loc].ProjectionX(hists_sim[iFile][type_loc].GetName()+f"sim_neg_proj){icentr}",ibiny0,ibiny1,ibinz0+5+antiproton,ibinz1+5+antiproton)
    Format_Hist_total(sim_neg_proj,Lcolor=1)
    pos_proj.Scale(1./(1+pos_proj.GetMaximum()))
    neg_proj.Scale(1./(1+neg_proj.GetMaximum()))
    sim_pos_proj.Scale(1./(1+sim_pos_proj.GetMaximum()))
    sim_neg_proj.Scale(1./(1+sim_neg_proj.GetMaximum()))
    pos_proj.Draw()
    neg_proj.Draw("same")
    sim_pos_proj.Draw("same")
    sim_neg_proj.Draw("same")
    legends.append(root.TLegend(0.2,0.5,0.5,0.9,f"{central_bins[2*icentr]}-{central_bins[2*icentr+1]}%"))
    legends[-1].AddEntry(pos_proj,legend_names[0],"l")
    legends[-1].AddEntry(neg_proj,legend_names[1],"l")
    legends[-1].AddEntry(sim_pos_proj,legend_names[2],"l")
    legends[-1].AddEntry(sim_neg_proj,legend_names[3],"l")
    Format_Legend(legends[-1])
    legends[-1].Draw()

c0.Draw()
c0.SaveAs(f"output/rich_emcal_qa/qa_pt_centr_{icentr}.png")

In [57]:
type_loc = 2
pt = [1.95,2.45]
c0 = root.TCanvas(f"c0",f"c0",1450,900)
c0.Divide(3,2)
projy = hists_read[0][type_loc].ProjectionY(f"projy{type_loc}")
projz = hists_read[0][type_loc].ProjectionZ(f"projz{type_loc}")
legends = []
for icentr in range(N_centr):
    c0.cd(1+icentr)
    ibiny0 = projy.FindBin(pt[0])
    ibiny1 = projy.FindBin(pt[1])-1
    ibinz0 = shift+projz.FindBin(central_bins[2*icentr+0])
    ibinz1 = shift+projz.FindBin(central_bins[2*icentr+1])-1
    pos_proj = hists_read[iFile][type_loc].ProjectionX(hists_read[iFile][type_loc].GetName()+f"pos_proj_{icentr}",ibiny0,ibiny1,ibinz0,ibinz1)
    Format_Hist_total(pos_proj,title_x="prob")
    neg_proj = hists_read[iFile][type_loc].ProjectionX(hists_read[iFile][type_loc].GetName()+f"neg_proj_{icentr}",ibiny0,ibiny1,ibinz0+5+antiproton,ibinz1+5+antiproton)
    Format_Hist_total(neg_proj,Lcolor=2,title_x="prob")
    sim_pos_proj = hists_sim[iFile][type_loc].ProjectionX(hists_sim[iFile][type_loc].GetName()+f"sim_pos_proj_{icentr}",ibiny0,ibiny1,ibinz0,ibinz1)
    Format_Hist_total(sim_pos_proj,Lcolor=root.kGreen+2)
    sim_neg_proj = hists_sim[iFile][type_loc].ProjectionX(hists_sim[iFile][type_loc].GetName()+f"sim_neg_proj){icentr}",ibiny0,ibiny1,ibinz0+5+antiproton,ibinz1+5+antiproton)
    Format_Hist_total(sim_neg_proj,Lcolor=1)
    pos_proj.Scale(1./(1+pos_proj.Integral()))
    neg_proj.Scale(1./(1+neg_proj.Integral()))
    sim_pos_proj.Scale(1./(1+sim_pos_proj.Integral()))
    sim_neg_proj.Scale(1./(1+sim_neg_proj.Integral()))
    neg_proj.SetMaximum(max(neg_proj.GetMaximum(),pos_proj.GetMaximum()))
    neg_proj.Draw()
    pos_proj.Draw("same")
    sim_pos_proj.Draw("same")
    sim_neg_proj.Draw("same")
    legends.append(root.TLegend(0.2,0.5,0.5,0.9,f"{central_bins[2*icentr]}-{central_bins[2*icentr+1]}%"))
    legends[-1].AddEntry(pos_proj,legend_names[0],"l")
    legends[-1].AddEntry(neg_proj,legend_names[1],"l")
    legends[-1].AddEntry(sim_pos_proj,legend_names[2],"l")
    legends[-1].AddEntry(sim_neg_proj,legend_names[3],"l")
    Format_Legend(legends[-1])
    legends[-1].Draw()

c0.Draw()
c0.SaveAs(f"output/rich_emcal_qa/qa_pt_centr_{icentr}.png")

In [58]:
type_loc = 1
pt = [1.95,2.45]
c0 = root.TCanvas(f"c0",f"c0",1450,900)
c0.Divide(3,2)
projy = hists_read[0][type_loc].ProjectionY(f"projy{type_loc}")
projz = hists_read[0][type_loc].ProjectionZ(f"projz{type_loc}")
legends = []
for icentr in range(N_centr):
    c0.cd(1+icentr)
    ibiny0 = projy.FindBin(pt[0])
    ibiny1 = projy.FindBin(pt[1])-1
    ibinz0 = shift+projz.FindBin(central_bins[2*icentr+0])
    ibinz1 = shift+projz.FindBin(central_bins[2*icentr+1])-1
    pos_proj = hists_read[iFile][type_loc].ProjectionX(hists_read[iFile][type_loc].GetName()+f"pos_proj_{icentr}",ibiny0,ibiny1,ibinz0,ibinz1)
    Format_Hist_total(pos_proj,title_x="n0")
    neg_proj = hists_read[iFile][type_loc].ProjectionX(hists_read[iFile][type_loc].GetName()+f"neg_proj_{icentr}",ibiny0,ibiny1,ibinz0+5+antiproton,ibinz1+5+antiproton)
    Format_Hist_total(neg_proj,Lcolor=2)
    sim_pos_proj = hists_sim[iFile][type_loc].ProjectionX(hists_sim[iFile][type_loc].GetName()+f"sim_pos_proj_{icentr}",ibiny0,ibiny1,ibinz0,ibinz1)
    Format_Hist_total(sim_pos_proj,Lcolor=root.kGreen+2)
    sim_neg_proj = hists_sim[iFile][type_loc].ProjectionX(hists_sim[iFile][type_loc].GetName()+f"sim_neg_proj){icentr}",ibiny0,ibiny1,ibinz0+5+antiproton,ibinz1+5+antiproton)
    Format_Hist_total(sim_neg_proj,Lcolor=1)
    pos_proj.Scale(1./(1+pos_proj.Integral()))
    neg_proj.Scale(1./(1+neg_proj.Integral()))
    sim_pos_proj.Scale(1./(1+sim_pos_proj.Integral()))
    sim_neg_proj.Scale(1./(1+sim_neg_proj.Integral()))
    pos_proj.SetMaximum(max(neg_proj.GetMaximum(),pos_proj.GetMaximum(),sim_pos_proj.GetMaximum(),sim_neg_proj.GetMaximum()))
    pos_proj.Draw()
    neg_proj.Draw("same")
    sim_pos_proj.Draw("same")
    sim_neg_proj.Draw("same")
    legends.append(root.TLegend(0.2,0.5,0.4,0.9,f"{central_bins[2*icentr]}-{central_bins[2*icentr+1]}%"))
    legends[-1].AddEntry(pos_proj,legend_names[0],"l")
    legends[-1].AddEntry(neg_proj,legend_names[1],"l")
    legends[-1].AddEntry(sim_pos_proj,legend_names[2],"l")
    legends[-1].AddEntry(sim_neg_proj,legend_names[3],"l")
    Format_Legend(legends[-1])
    legends[-1].Draw()

c0.Draw()
c0.SaveAs(f"output/rich_emcal_qa/qa_pt_centr_{icentr}.png")

In [59]:
type_loc = 3
pt = [1.95,2.45]
c0 = root.TCanvas(f"c0",f"c0",1450,900)
c0.Divide(3,2)
projy = hists_read[0][type_loc].ProjectionY(f"projy{type_loc}")
projz = hists_read[0][type_loc].ProjectionZ(f"projz{type_loc}")
legends = []
for icentr in range(N_centr):
    c0.cd(1+icentr)
    ibiny0 = projy.FindBin(pt[0])
    ibiny1 = projy.FindBin(pt[1])-1
    ibinz0 = shift+projz.FindBin(central_bins[2*icentr+0])
    ibinz1 = shift+projz.FindBin(central_bins[2*icentr+1])-1
    pos_proj = hists_read[iFile][type_loc].ProjectionX(hists_read[iFile][type_loc].GetName()+f"pos_proj_{icentr}",ibiny0,ibiny1,ibinz0,ibinz1)
    Format_Hist_total(pos_proj,title_x="disp")
    neg_proj = hists_read[iFile][type_loc].ProjectionX(hists_read[iFile][type_loc].GetName()+f"neg_proj_{icentr}",ibiny0,ibiny1,ibinz0+5+antiproton,ibinz1+5+antiproton)
    Format_Hist_total(neg_proj,Lcolor=2)
    sim_pos_proj = hists_sim[iFile][type_loc].ProjectionX(hists_sim[iFile][type_loc].GetName()+f"sim_pos_proj_{icentr}",ibiny0,ibiny1,ibinz0,ibinz1)
    Format_Hist_total(sim_pos_proj,Lcolor=root.kGreen+2)
    sim_neg_proj = hists_sim[iFile][type_loc].ProjectionX(hists_sim[iFile][type_loc].GetName()+f"sim_neg_proj){icentr}",ibiny0,ibiny1,ibinz0+5+antiproton,ibinz1+5+antiproton)
    Format_Hist_total(sim_neg_proj,Lcolor=1)
    pos_proj.Scale(1./(1+pos_proj.Integral()))
    neg_proj.Scale(1./(1+neg_proj.Integral()))
    sim_pos_proj.Scale(1./(1+sim_pos_proj.Integral()))
    sim_neg_proj.Scale(1./(1+sim_neg_proj.Integral()))
    pos_proj.SetMaximum(max(neg_proj.GetMaximum(),pos_proj.GetMaximum(),sim_pos_proj.GetMaximum(),sim_neg_proj.GetMaximum()))
    pos_proj.Draw()
    neg_proj.Draw("same")
    sim_pos_proj.Draw("same")
    sim_neg_proj.Draw("same")
    legends.append(root.TLegend(0.5,0.5,0.9,0.9,f"{central_bins[2*icentr]}-{central_bins[2*icentr+1]}%"))
    legends[-1].AddEntry(pos_proj,legend_names[0],"l")
    legends[-1].AddEntry(neg_proj,legend_names[1],"l")
    legends[-1].AddEntry(sim_pos_proj,legend_names[2],"l")
    legends[-1].AddEntry(sim_neg_proj,legend_names[3],"l")
    legends[-1].Draw()

c0.Draw()
c0.SaveAs(f"output/rich_emcal_qa/qa_pt_centr_{icentr}.png")

In [60]:
type_loc = 4
pt = [1.95,2.45]
c0 = root.TCanvas(f"c0",f"c0",1450,900)
c0.Divide(3,2)
projy = hists_read[0][type_loc].ProjectionY(f"projy{type_loc}")
projz = hists_read[0][type_loc].ProjectionZ(f"projz{type_loc}")
legends = []
for icentr in range(N_centr):
    c0.cd(1+icentr)
    ibiny0 = projy.FindBin(pt[0])
    ibiny1 = projy.FindBin(pt[1])-1
    ibinz0 = shift+projz.FindBin(central_bins[2*icentr+0])
    ibinz1 = shift+projz.FindBin(central_bins[2*icentr+1])-1
    pos_proj = hists_read[iFile][type_loc].ProjectionX(hists_read[iFile][type_loc].GetName()+f"pos_proj_{icentr}",ibiny0,ibiny1,ibinz0,ibinz1)
    Format_Hist_total(pos_proj,title_x="#chi^{2}/npe0")
    neg_proj = hists_read[iFile][type_loc].ProjectionX(hists_read[iFile][type_loc].GetName()+f"neg_proj_{icentr}",ibiny0,ibiny1,ibinz0+5+antiproton,ibinz1+5+antiproton)
    Format_Hist_total(neg_proj,Lcolor=2)
    sim_pos_proj = hists_sim[iFile][type_loc].ProjectionX(hists_sim[iFile][type_loc].GetName()+f"sim_pos_proj_{icentr}",ibiny0,ibiny1,ibinz0,ibinz1)
    Format_Hist_total(sim_pos_proj,Lcolor=root.kGreen+2)
    sim_neg_proj = hists_sim[iFile][type_loc].ProjectionX(hists_sim[iFile][type_loc].GetName()+f"sim_neg_proj){icentr}",ibiny0,ibiny1,ibinz0+5+antiproton,ibinz1+5+antiproton)
    Format_Hist_total(sim_neg_proj,Lcolor=1)
    pos_proj.Scale(1./(1+pos_proj.GetMaximum()))
    neg_proj.Scale(1./(1+neg_proj.GetMaximum()))
    sim_pos_proj.Scale(1./(1+sim_pos_proj.GetMaximum()))
    sim_neg_proj.Scale(1./(1+sim_neg_proj.GetMaximum()))
    pos_proj.Draw()
    neg_proj.Draw("same")
    sim_pos_proj.Draw("same")
    sim_neg_proj.Draw("same")
    legends.append(root.TLegend(0.5,0.5,0.9,0.9,f"{central_bins[2*icentr]}-{central_bins[2*icentr+1]}%"))
    legends[-1].AddEntry(pos_proj,legend_names[0],"l")
    legends[-1].AddEntry(neg_proj,legend_names[1],"l")
    legends[-1].AddEntry(sim_pos_proj,legend_names[2],"l")
    legends[-1].AddEntry(sim_neg_proj,legend_names[3],"l")
    Format_Legend(legends[-1])
    legends[-1].Draw()

c0.Draw()
c0.SaveAs(f"output/rich_emcal_qa/qa_pt_centr_{icentr}.png")

In [61]:
type_loc = 5
pt = [1.95,2.45]
c0 = root.TCanvas(f"c0",f"c0",1450,900)
c0.Divide(3,2)
projy = hists_read[0][type_loc].ProjectionY(f"projy{type_loc}")
projz = hists_read[0][type_loc].ProjectionZ(f"projz{type_loc}")
legends = []
for icentr in range(N_centr):
    c0.cd(1+icentr)
    ibiny0 = projy.FindBin(pt[0])
    ibiny1 = projy.FindBin(pt[1])-1
    ibinz0 = shift+projz.FindBin(central_bins[2*icentr+0])
    ibinz1 = shift+projz.FindBin(central_bins[2*icentr+1])-1
    pos_proj = hists_read[iFile][type_loc].ProjectionX(f"pos_proj_{icentr}",ibiny0,ibiny1,ibinz0,ibinz1)
    Format_Hist_total(pos_proj,title_x="m_tof")
    neg_proj = hists_read[iFile][type_loc].ProjectionX(f"neg_proj_{icentr}",ibiny0,ibiny1,ibinz0+5+antiproton,ibinz1+5+antiproton)
    Format_Hist_total(neg_proj,Lcolor=2,title_x="m2_tof")
    sim_pos_proj = hists_sim[iFile][type_loc].ProjectionX(f"sim_pos_proj_{icentr}",ibiny0,ibiny1,ibinz0,ibinz1)
    Format_Hist_total(sim_pos_proj,Lcolor=root.kGreen+2)
    sim_neg_proj = hists_sim[iFile][type_loc].ProjectionX(f"sim_neg_proj){icentr}",ibiny0,ibiny1+5)
    pos_proj.Rebin(10)
    neg_proj.Rebin(10)
    Format_Hist_total(sim_neg_proj,Lcolor=1)
    pos_proj.Scale(1./(1+pos_proj.Integral()))
    neg_proj.Scale(1./(1+neg_proj.Integral()))
    neg_proj.Draw()
    pos_proj.SetLineStyle(2)
    pos_proj.Draw("same")
    #sim_pos_proj.Draw("same")
    #sim_neg_proj.Draw("same")
    legends.append(root.TLegend(0.5,0.5,0.9,0.9,f"{central_bins[2*icentr]}-{central_bins[2*icentr+1]}%"))
    legends[-1].AddEntry(pos_proj,"pos","l")
    legends[-1].AddEntry(neg_proj,"neg","l")
    #legends[-1].AddEntry(sim_pos_proj,"sim pos","l")
    #legends[-1].AddEntry(sim_neg_proj,"sim neg","l")
    Format_Legend(legends[-1])
    legends[-1].Draw()
    print(round(100*pos_proj.Integral(pos_proj.FindBin(0.7),pos_proj.FindBin(1.4))/pos_proj.Integral(pos_proj.FindBin(-0.3),pos_proj.FindBin(0.4)),1),\
          round(100*neg_proj.Integral(neg_proj.FindBin(0.7),neg_proj.FindBin(1.4))/neg_proj.Integral(neg_proj.FindBin(-0.3),neg_proj.FindBin(0.4)),1))

c0.Draw()
c0.SaveAs(f"output/rich_emcal_qa/qa_pt_centr_{icentr}.png")

3.8 16.3
4.1 20.9
3.5 11.5
2.7 6.3
2.2 2.8
1.1 1.1


In [62]:
type_loc = 6
pt = [1.45,2.45]
c0 = root.TCanvas(f"c0",f"c0",1450,900)
c0.Divide(3,2)
projy = hists_read[0][type_loc].ProjectionY(f"projy{type_loc}")
projz = hists_read[0][type_loc].ProjectionZ(f"projz{type_loc}")
legends = []
for icentr in range(N_centr):
    c0.cd(1+icentr)
    ibiny0 = projy.FindBin(pt[0])
    ibiny1 = projy.FindBin(pt[1])-1
    ibinz0 = shift+projz.FindBin(central_bins[2*icentr+0])
    ibinz1 = shift+projz.FindBin(central_bins[2*icentr+1])-1
    pos_proj = hists_read[iFile][type_loc].ProjectionX(f"pos_proj_{icentr}",ibiny0,ibiny1,ibinz0,ibinz1)
    Format_Hist_total(pos_proj,title_x="temc")
    neg_proj = hists_read[iFile][type_loc].ProjectionX(f"neg_proj_{icentr}",ibiny0,ibiny1,ibinz0+5+antiproton,ibinz1+5+antiproton)
    Format_Hist_total(neg_proj,Lcolor=2)
    sim_pos_proj = hists_sim[iFile][type_loc].ProjectionX(f"sim_pos_proj_{icentr}",ibiny0,ibiny1,ibinz0,ibinz1)
    Format_Hist_total(sim_pos_proj,Lcolor=root.kGreen+2)
    sim_neg_proj = hists_sim[iFile][type_loc].ProjectionX(f"sim_neg_proj){icentr}",ibiny0,ibiny1,ibinz0+5+antiproton,ibinz1+5+antiproton)
    Format_Hist_total(sim_neg_proj,Lcolor=1)
    pos_proj.Scale(1./(1+pos_proj.GetMaximum()))
    neg_proj.Scale(1./(1+neg_proj.GetMaximum()))
    sim_pos_proj.Scale(1./(1+sim_pos_proj.GetMaximum()))
    sim_neg_proj.Scale(1./(1+sim_neg_proj.GetMaximum()))
    pos_proj.Draw()
    neg_proj.Draw("same")
    #sim_pos_proj.Draw("same")
    #sim_neg_proj.Draw("same")
    legends.append(root.TLegend(0.5,0.5,0.9,0.9,f"{central_bins[2*icentr]}-{central_bins[2*icentr+1]}%"))
    legends[-1].AddEntry(pos_proj,"pos","l")
    legends[-1].AddEntry(neg_proj,"neg","l")
    #legends[-1].AddEntry(sim_pos_proj,"sim pos","l")
    #legends[-1].AddEntry(sim_neg_proj,"sim neg","l")
    Format_Legend(legends[-1])
    legends[-1].Draw()

c0.Draw()
c0.SaveAs(f"output/rich_emcal_qa/qa_pt_centr_{icentr}.png")

In [63]:
type_loc = 10
pt = [1.45,2.45]
c0 = root.TCanvas(f"c0",f"c0",1450,900)
c0.Divide(3,2)
projx = hists_read[0][type_loc].ProjectionY(f"projy{type_loc}")
projz = hists_read[0][type_loc].ProjectionZ(f"projz{type_loc}")
legends = []
for icentr in range(N_centr):
    c0.cd(1+icentr)
    ibinx0 = projx.FindBin(0)
    ibinx1 = projx.FindBin(10)-1
    ibinz0 = shift+projz.FindBin(central_bins[2*icentr+0])
    ibinz1 = shift+projz.FindBin(central_bins[2*icentr+1])-1
    pos_proj = hists_read[iFile][type_loc].ProjectionY(f"pos_proj_{icentr}",ibinx0,ibinx1,ibinz0,ibinz1)
    Format_Hist_total(pos_proj,title_x="npe0")
    sim_pos_proj = hists_sim[iFile][type_loc].ProjectionY(f"sim_pos_proj_{icentr}",ibinx0,ibinx1,ibinz0,ibinz1)
    Format_Hist_total(sim_pos_proj,Lcolor=root.kGreen+2)
    pos_proj.Scale(1./(1+pos_proj.GetMaximum()))
    sim_pos_proj.Scale(1./(1+sim_pos_proj.GetMaximum()))
    pos_proj.Draw()
    sim_pos_proj.Draw("same")
    legends.append(root.TLegend(0.5,0.5,0.9,0.9,f"{central_bins[2*icentr]}-{central_bins[2*icentr+1]}%"))
    legends[-1].AddEntry(pos_proj,"data","l")
    legends[-1].AddEntry(sim_pos_proj,"sim","l")
    Format_Legend(legends[-1])
    legends[-1].Draw()

c0.Draw()
c0.SaveAs(f"output/rich_emcal_qa/qa_pt_centr_{icentr}.png")